# High Performance Jupyter

## GPU cluster!!!

|<img src="https://rapids.ai/assets/images/RAPIDS-logo-purple.svg" width="400">| <img src="https://docs.dask.org/en/latest/_images/dask_horizontal.svg" width="300">|
|--|--|

We will do the same analysis as [rapids.ipynb](rapids.ipynb), except we will be using a roughly 10x larger dataset on a cluster of GPU machines.

AWS EC2 instances: 3 g4dn.xlarge (NVIDIA T4 CPU, 16GB GPU RAM)

This notebook combines all the lessons we learned from using Dask and RAPIDS. Now we're operating in a cluster environment, except the nodes in the cluster are using GPUs to accelerate the computations. 

In [1]:
from dask.distributed import Client
from dask_saturn import SaturnCluster

n_workers = 3
cluster = SaturnCluster(
    n_workers=n_workers, 
    scheduler_size='medium',
    worker_size='g4dnxlarge',
)
client = Client(cluster)
cluster

[2020-09-26 20:41:44] INFO - dask-saturn | Cluster is ready


There are a couple of other dashboard pages worth viewing for GPU memory and utilization that are not listed on the navbar, so we grab direct links for those below.

In [2]:
from IPython.display import display, HTML

gpu_links = f'''
<b>GPU Dashboard links</b>
<ul>
<li><a href="{client.dashboard_link}/individual-gpu-memory" target="_blank">GPU memory</a></li>
<li><a href="{client.dashboard_link}/individual-gpu-utilization" target="_blank">GPU utilization</a></li>
</ul>
'''
display(HTML(gpu_links))

<br>The scheduler might be ready before all the workers are. We'll wait until all the workers are up.

In [3]:
client.wait_for_workers(3)

In [4]:
# dask_cudf is the RAPIDS dataframe library with Dask integration
import dask_cudf as cudd
import numpy as np
import datetime
import s3fs
import warnings
warnings.simplefilter("ignore")

data_path = 's3://nyc-tlc/trip data'
seed = 42

# Load and explore data

Notice the glob syntax for the filename argument, which tells Dask to load all files with this pattern.

In [5]:
%%time

taxi = cudd.read_csv(
    f'{data_path}/yellow_tripdata_2019-*.csv',
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
    assume_missing=True,
    storage_options={'anon': True},
)

CPU times: user 917 ms, sys: 401 ms, total: 1.32 s
Wall time: 1.71 s


In [6]:
%%time
print(f"Row count: {len(taxi)}")
print(f"Size in GB: {taxi.memory_usage(deep=True).sum().compute() / 1e9}")

Row count: 84399019
Size in GB: 11.904947958
CPU times: user 157 ms, sys: 19.2 ms, total: 176 ms
Wall time: 32.6 s


In [7]:
# doesn't work as of dask_cudf version 0.14
# taxi.describe().compute().T

# Feature engineering

Same feature engineering from [laptop.ipynb](laptop.ipynb), using the same code!

In [8]:
numeric_feat = [
    'pickup_weekday', 
    'pickup_hour', 
    'pickup_week_hour', 
    'pickup_minute', 
    'passenger_count',
]
categorical_feat = [
    'PULocationID', 
    'DOLocationID',
]
features = numeric_feat + categorical_feat
y_col = 'high_tip'

In [9]:
def prep_df(df: cudd.DataFrame) -> cudd.DataFrame:
    '''
    Generate features from a raw taxi dataframe.
    Use 32 bit precision for GPU processing
    '''
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    df['tip_fraction'] = df.tip_amount / df.fare_amount
    df['high_tip'] = (df['tip_fraction'] > 0.2) # class label
    
    df['pickup_weekday'] = df.tpep_pickup_datetime.dt.weekday
    # as of version 0.15, cudf doesn't support weekofyear
    # df['pickup_weekofyear'] = df.tpep_pickup_datetime.dt.weekofyear
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['pickup_minute'] = df.tpep_pickup_datetime.dt.minute
    df = df[features + [y_col]].astype('float32').fillna(-1)
    df[y_col] = df[y_col].astype('int32')
    
    return df
    
taxi = prep_df(taxi)

In [10]:
taxi.head()

,pickup_weekday,pickup_hour,pickup_week_hour,pickup_minute,passenger_count,PULocationID,DOLocationID,high_tip
0,1.0,0.0,24.0,46.0,1.0,151.0,239.0,1
1,1.0,0.0,24.0,59.0,1.0,239.0,246.0,0
2,4.0,13.0,109.0,48.0,3.0,236.0,236.0,0
3,2.0,15.0,63.0,52.0,5.0,193.0,193.0,0
4,2.0,15.0,63.0,56.0,5.0,193.0,193.0,0


In [11]:
%%time
from dask.distributed import wait

taxi = taxi.persist()
_ = wait(taxi)

CPU times: user 229 ms, sys: 2.34 ms, total: 231 ms
Wall time: 17.6 s


## Random forest

We're training the same model as [rapids.ipynb](rapids.ipynb), except with 10x more data.

As of version 0.14, RAPIDS doesn't have a Dask (distributed) implementation of `train_test_split`, so we'll train using the full 2019 dataset and use a few months from 2020 as a test set. 

In [12]:
from cuml.dask.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(
    n_estimators=100, 
    max_depth=5, 
    seed=seed,
)

In [13]:
%%time
_ = rfc.fit(taxi[features], taxi[y_col])

CPU times: user 271 ms, sys: 104 ms, total: 375 ms
Wall time: 3.82 s


In [14]:
taxi_test = cudd.read_csv(
    # first 3 months of 2020
    [f'{data_path}/yellow_tripdata_2020-0{m}.csv' for m in [1, 2, 3]],
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
    assume_missing=True,
    storage_options={'anon': True},
)
taxi_test = prep_df(taxi_test)

As of version 0.14, RAPIDS doesn't have a distributed implementation of `roc_auc_score`, so we'll use `compute()` to pull data down to a single GPU.

In [15]:
from cuml.metrics import roc_auc_score

preds = rfc.predict_proba(taxi_test[features])[1]
roc_auc_score(taxi_test[y_col].compute(), preds.compute())

array(0.5368982, dtype=float32)